In [1]:
import numpy as np
import pandas as pd
import math
np.set_printoptions(suppress=True)

In [2]:
user_artist_df = pd.read_csv('user_artist.csv')
test_df = pd.read_csv('test.csv')

In [3]:
new_df = (user_artist_df.copy())
new_test_df = (test_df.copy())
# mask = (new_test_df['userID'] == 145)
# mask1 = (new_test_df['userID'] == 145) | (new_test_df['userID'] == 1539) | (new_test_df['userID'] == 2214)| (new_test_df['userID'] == 211) | (new_test_df['userID'] == 3028)
# new_test_df = new_test_df[mask] 
new_test_df['weight'] = -1

In [4]:
new_test_df1 = pd.pivot_table(new_test_df, values = 'weight', index = ['userID'], columns = ['artistID'])

In [5]:
new_df1 = pd.pivot_table(new_df, values="weight", index=['userID'], columns=["artistID"])

In [6]:
new_df1.fillna(0, inplace=True)

In [7]:
# R avg 
r_avg = (pd.DataFrame(new_df1).to_numpy()).mean()
r_avg

1.8910221059478527

In [8]:
result = pd.concat([new_df1, new_test_df1])
# res1 = result.select_dtypes(np.number).max(level=0)
res2 = result.groupby(level = 0).median()

cols = sorted(list(res2.columns))
sortes3 = res2[cols]
sortes3.fillna(-2, inplace=True)

In [9]:
mtrx_R = pd.DataFrame(sortes3).to_numpy()

In [10]:
def calc_deviation_b(arr):
    n = len(arr[arr >= 0])
    if n == 0:
        return 0
    sum_arr = 0
    for i in range(len(arr)):
        if arr[i] >= 0:
            sum_arr += arr[i]
    return sum_arr/n

In [11]:
b_u = []
b_i = []
for i in range(len(mtrx_R)):
    b_u.append((calc_deviation_b(mtrx_R[i]) - r_avg))

mtrx_R_tr = mtrx_R.T

for i in range(len(mtrx_R_tr)):
    b_i.append((calc_deviation_b(mtrx_R_tr[i]) - r_avg))

In [12]:
# create R_hat
to_filled = []
mtrx_R_hat = np.zeros((len(mtrx_R), len(mtrx_R[0])))
for i in range(len(mtrx_R)):
     for j in range(len(mtrx_R[0])):
        # here don't use -1 yet.
        if mtrx_R[i,j] == -2:
            mtrx_R_hat[i,j] = 0
        else:
            mtrx_R_hat[i,j] = r_avg + b_u[i] + b_i[j]
            if mtrx_R[i,j] == -1:
                to_filled.append((i,j))
        
            

In [13]:
# create R_tilda
mtrx_R_tilda = mtrx_R - mtrx_R_hat

# put -1 after the sub.

for (i,j) in to_filled:
    mtrx_R_tilda[i,j] = -1


In [14]:
def special_inner_profuct(mtrx):
    mtrx_transpose = mtrx.T
    mtrx_R_transpose_copy = mtrx_transpose.copy()
    mtrx_R_transpose_copy = np.where(mtrx_R_transpose_copy == -1, 0, mtrx_R_transpose_copy)
    mtrx_R_transpose_copy = np.where(mtrx_R_transpose_copy == -2, 0, mtrx_R_transpose_copy)
    D = np.zeros((len(mtrx[0]), len(mtrx[0])))
    
    for i in range(len(mtrx_R_transpose_copy)):
        for j in range(len(mtrx_R_transpose_copy)):
            mone = np.dot(mtrx_R_transpose_copy[i], mtrx_R_transpose_copy[j])
            x = 0
            y = 0
            for k in range(len(mtrx_R_transpose_copy[0])):
                
                if mtrx_R_transpose_copy[i][k] != -1 and mtrx_R_transpose_copy[j][k] != -1 and mtrx_R_transpose_copy[i][k] != -2 and mtrx_R_transpose_copy[j][k] != -2:
                    x += (mtrx_R_transpose_copy[i][k] ** 2)
                    y += (mtrx_R_transpose_copy[j][k] ** 2)     
            norm1 = x ** 0.5
            norm2 = y ** 0.5
            
            mechane = norm1 * norm2
            if mechane == 0:
                D[i,j] = 0
            else:
                D[i,j] = mone / mechane
    
#     for i in range(len(mtrx)):
#         print(mtrx[i])
#     return
            
    return D

In [15]:
def get_nearest(similarities, n):
    # n is the number of neighboors we want.
    
    sim_arr = []
    
    for i in range(len(similarities)):
        array = similarities[i].copy()
        array_non_neg = array.copy()
        for j in range(len(array)):
            if int(array_non_neg[j]) == 1 or int(array_non_neg[j]) == -1:
                array_non_neg[j] = 0
            if array_non_neg[j] < 0:
                array_non_neg[j] = array_non_neg[j] * (-1)

        ind = np.argpartition(array_non_neg, -n)[-n:]
        
        top2 = array[sorted(ind)]
        sim_arr.append((top2, sorted(ind)))

    
    return sim_arr

In [16]:
def get_final_R(mtrx_R_hat, mtrx_R_tilda, similiraties):
    nearest = get_nearest(similarities, 2) # get the 2 nearest movies and their indexes.
   
    mtrx_R_final = np.zeros((len(mtrx_R_hat), len(mtrx_R_hat[0])))

    for i in range(len(mtrx_R_hat)):
        for j in range(len(mtrx_R_hat[0])):
#             if (i,j) != (0,1) and (i,j) != (1,0): # this will be replaced by the indexes we extract (cells which need to be filled)
#                 mtrx_R_final[i,j] = mtrx_R_hat[i,j] # only predict for the missing ones.
#                 continue
            
            most_similar_mvs_to_j = nearest[j][1] # similarity indexes.
            r_tilda  = (mtrx_R_tilda[i][most_similar_mvs_to_j])
            mone1 = np.dot(nearest[j][0], r_tilda)
            
            mechane1 = 0
            for k in range(len(nearest[j][0])):
                mechane1 += abs(nearest[j][0][k])
                
            if mechane1 == 0:
                mtrx_R_final[i,j] = mtrx_R_hat[i,j]
            else:
                mtrx_R_final[i,j] = mtrx_R_hat[i,j] + mone1/mechane1
    return mtrx_R_final

In [ ]:
similarities = special_inner_profuct(mtrx_R_tilda)
m = get_final_R(mtrx_R_hat, mtrx_R_tilda, similarities)
m

In [ ]:
mtrx_R